In [2]:
from core import *

C:\Users\tikyn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [ ]:

#98 per un solo video

# 1. model setting - in: 98 timeseries - out: 98 prediction
# 2. splittare in 6 salienze - in: 98 prediction - out: 98 into 6 list[salienza][feature][elemento singolo] (could be repetition)
# 3xyz. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 3tog. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 4. unione - in: 6 list[][] - out: list[]
# 4.1 ignorezone(margin)
# 5. delnear(margin)
# 6. dtw(threshold)
# Filtro(pred, maj_margin, maj_thre, zone_margin, del_margin, dtw_threshold) -> final_pred




In [ ]:
#leave one video out

param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}


# 21*98 = 2058 timeseries
for p_idx,params in enumerate(ParameterGrid(param_grid)):

    for i in range(len(TIMESERIES)):
        train_set = [j for j in range(len(TIMESERIES)) if j != i]
        test_set = i

        for train in train_set:

    #fuori dal for deve calcolare la media
    

    

# for each train set
## 

for i in range[0,3]: #threshold
    for j in range[0,5]: #video
        model(i,j)

# calcolo 1 valore di threshold per tutti i video e faccio media
# calcolo 2 valore di threshold per tutti i video e faccio media
# scelgo lo score migliore tra i due e lo applico al test set e ottengo risultato
# oppure
# calcolo 1 valore di threshold per tutti i video 

# 1 setting -> cross validation
# 2 setting -> cross validation

# fit() su train
# predict() su test -> score
##
# per ogni fold fit() e predict()
# media dei predict() -> score


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
0
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
1
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
2
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
3
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
4
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
5
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
6
[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
7
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
8
[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
10
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20]
11
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20]
12
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,